In [1]:
from augur.utils import json_to_tree
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import itertools
import requests
import collections
import math
import json

%matplotlib inline


Bad key "text.kerning_factor" on line 4 in
/Users/katekistler/anaconda3/envs/nextstrain/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
#Download tree json
tree_url = "https://data.nextstrain.org/ncov_global.json"

tree_json = requests.get(tree_url).json()

#Put tree in Bio.Phylo format
tree_ncov = json_to_tree(tree_json)

In [249]:
#for testing
with open('../../seasonal-cov/229e/auspice/seasonal_corona_229e_spike.json') as json_handle:
    json_tree_229e = json.load(json_handle)

tree_229e = json_to_tree(json_tree_229e)

In [3]:
#for bhatt clades on Rhino... but can't use augur there

def find_clades(tree):
    
    all_clades_and_parents = []
    
    for node in tree.find_clades():

        if not node.is_terminal():
            children = [child.name for child in node.get_terminals()]
            all_clades_and_parents.append({'parent': node.name, 'children': children})

    
    with open('all_clades_and_parents.json', 'w') as fout:
        json.dump(all_clades_and_parents, fout)
    
#     return all_clades_and_parents

In [4]:
clade_nodes_by_name = {}
for node in tree.find_clades(terminal=False):
    if "clade_membership" in node.node_attrs:
        clade_name = node.node_attrs["clade_membership"]["value"]

        clade_nodes_by_name[node.name] = clade_name
        
with open('clades_by_node_name.json', 'w') as file_out:
    json.dump(clade_nodes_by_name, file_out)

NameError: name 'tree' is not defined

In [322]:
find_clades(tree_ncov)

In [12]:
def tally_individual_mut_occurence(tree):
    all_spike_mutation_occurences = []
    tree_tips = tree.get_terminals()
    tree_internals = tree.get_nonterminals()
    all_tree_nodes = tree_tips+tree_internals
    
    num_branchs = len(all_tree_nodes)

    for node in all_tree_nodes:
        if len(node.get_terminals())>=10:
            if hasattr(node, "branch_attrs") and "mutations" in node.branch_attrs:
                if "S" in node.branch_attrs["mutations"]:   
                    all_spike_mutation_occurences += node.branch_attrs["mutations"]["S"]
    
    #create count of number of times each spike mutation has happened on tree
    spike_mut_count = collections.Counter(all_spike_mutation_occurences)
    
    #normalize this by number of branchs on tree
    spike_mut_freq = {x:y/num_branchs for x,y in spike_mut_count.items()}
    
    return spike_mut_count, spike_mut_freq

In [11]:
def get_parent(tree, child_clade):
    node_path = tree.get_path(child_clade)
    return node_path

def tally_pair_occurences(tree):
    
    #all paths from root to tip
    all_paths = []
    

    #dictionary spike muts at each branch
    all_branchs_spikemuts = {}

    for node in tree.find_clades():
        if len(node.get_terminals())>=10:
            node_path = get_parent(tree, node)
            #path from root to tip, designated by node names
            node_names_path = [x.name for x in node_path]
            #append this to a list of all paths from root to tip
            all_paths.append(node_names_path)

            #find spike muts along this path
            path_spikemuts = []
            for y in node_path:
                if hasattr(y, "branch_attrs") and "mutations" in y.branch_attrs:
                    if "S" in y.branch_attrs["mutations"]:   
                        if y.name not in all_branchs_spikemuts.keys():
                            all_branchs_spikemuts[y.name] = y.branch_attrs["mutations"]["S"]

        
        
    #keep track of paths on the tree already considered
    traveled_segments = []
    
    #find all pairs of segments that occur along path of tree, without re-counting basal segments of the path
    all_branch_pairs = []
    
    for path in all_paths:
        untraveled_segments = []
        #find portion of the path that has already been traveled, and new portion
        for y in range(len(path)):
            while path[:y+1] not in traveled_segments:
                traveled_segments.append(path[:y+1])
                untraveled_segments+=path[y:]
        

        #make pairs between only the new segments of path with all other parts of path
        untraveled_segment_pairs = itertools.product(untraveled_segments, path)
        
        for pair in untraveled_segment_pairs:
            if pair[0] != pair [1]:
                all_branch_pairs.append(pair)
                
    #convert branch pairs into mutation pairs, some branchs may have multiple mutations, 
    #leading to multiple mutation pairs for that branch pair   
    all_mutation_pairs = []
    
    for branch_pair in all_branch_pairs:
        #only care if both branchs have spike mutations on them
        if branch_pair[0] in all_branchs_spikemuts.keys() and branch_pair[1] in all_branchs_spikemuts.keys():
            muts_branch0 = all_branchs_spikemuts[branch_pair[0]]
            muts_branch1 = all_branchs_spikemuts[branch_pair[1]]
            muts_pairs = itertools.product(muts_branch0, muts_branch1)
            for mut_pair in muts_pairs:
                all_mutation_pairs.append(mut_pair)
    
    #add pairs that occur on same branch
    for k,v in all_branchs_spikemuts.items():
        if len(v)>=2:
            branch_pairs = itertools.combinations(v, 2)
            for bp in branch_pairs:
                all_mutation_pairs.append(bp)
            


    #number of paths from root to tip
    num_paths = len(all_paths)
    
    count_mutation_pairs = collections.Counter(all_mutation_pairs)
    
    spike_pairs_freq = {x:y/num_paths for x,y in count_mutation_pairs.items()}


    return count_mutation_pairs, spike_pairs_freq 
    

In [7]:
def calc_enrichment_freq(tree, pair_min = 2):
    
    spike_mut_count, spike_mut_freq = tally_individual_mut_occurence(tree)
    count_mutation_pairs, spike_pairs_freq = tally_pair_occurences(tree)
    
    enrichment_scores = []
    
    #not sure if this is the right way to do this
    for pair, pair_freq in spike_pairs_freq.items():
        #only consider pairs that occur at least pair_min times on tree
        if count_mutation_pairs[pair] >= pair_min:

            mut0 = pair[0]
            mut1 = pair[1]
            freq0 = spike_mut_freq[mut0]
            freq1 = spike_mut_freq[mut1]

            enrichment = pair_freq / (freq0*freq1)
            enrichment_scores.append({'pair': pair, 'enrichment_score': enrichment, 'pair_freq': pair_freq})
    
    enrichment_df = pd.DataFrame(enrichment_scores)
    print(enrichment_df.nlargest(20, 'enrichment_score'))
        
    
    

In [13]:
calc_enrichment_freq(tree_ncov)

              pair  enrichment_score  pair_freq
9   (G142D, E484Q)      55037.284615   0.005495
18   (G142D, T19R)      44029.827692   0.004396
19  (G142D, E156-)      44029.827692   0.004396
20  (G142D, F157-)      44029.827692   0.004396
21  (G142D, R158G)      44029.827692   0.004396
24  (G142D, Q484E)      44029.827692   0.004396
58    (L5F, A570D)      36691.523077   0.002198
61    (L5F, S982A)      36691.523077   0.002198
0   (N439K, D614G)      27518.642308   0.002198
1    (H69-, D614G)      27518.642308   0.004396
2    (V70-, D614G)      27518.642308   0.004396
3   (P681H, D614G)      27518.642308   0.007692
16  (D950N, D614G)      27518.642308   0.002198
17  (T478K, D614G)      27518.642308   0.002198
25  (A222V, D614G)      27518.642308   0.002198
34  (Y144-, D614G)      27518.642308   0.004396
35  (E484K, D614G)      27518.642308   0.008791
37  (T716I, D614G)      27518.642308   0.002198
38   (P26S, D614G)      27518.642308   0.002198
39  (L241-, D614G)      27518.642308   0

In [14]:
def calc_enrichment_count(tree, ind_min=3, pair_min=3):
    
    spike_mut_count, spike_mut_freq = tally_individual_mut_occurence(tree)
    count_mutation_pairs, spike_pairs_freq = tally_pair_occurences(tree)
    
    enrichment_scores = []
    
    #not sure if this is the right way to do this
    for pair, pair_count in count_mutation_pairs.items():
        mut0 = pair[0]
        mut1 = pair[1]
        count0 = spike_mut_count[mut0]
        count1 = spike_mut_count[mut1]
        
        #only consider pairs that were observed at least pair_min number of times
        #and where the individual mutations occurred at least ind_min number of times accross the tree
        if pair_count>=pair_min:
            if count0 >= ind_min and count1 >= ind_min:
                enrichment = pair_count / (count0*count1)
                enrichment_scores.append({'pair': pair, 'enrichment_score': enrichment,
                                          'pair_count': pair_count, 'scoring': 'together',
                                          'individual_mut_count0': count0, 'individual_mut_count1': count1})
#             if count0 >= ind_min:
#                 enrichment0 = pair_count / count0
#                 if mut1!= 'D614G':
#                     enrichment_scores.append({'pair': pair, 'enrichment_score': enrichment0,
#                                               'pair_count': pair_count, 'scoring': '0only',
#                                               'individual_mut': mut0, 'individual_mut_count': count0})
#             if count1 >= ind_min:
#                 enrichment1 = pair_count / count1
#                 enrichment_scores.append({'pair': pair, 'enrichment_score': enrichment1, 'scoring': '1only',
#                                           'pair_count': pair_count, 'individual_mut': mut1, 
#                                           'individual_mut_count': count1})
    
    enrichment_df = pd.DataFrame(enrichment_scores)
    print(enrichment_df.nlargest(50, 'enrichment_score'))
        
    
    

In [15]:
calc_enrichment_count(tree_ncov)

             pair  enrichment_score  pair_count   scoring  \
1  (G142D, P681R)          0.333333           5  together   
5    (H69-, V70-)          0.250000           4  together   
0  (G142D, L452R)          0.200000           5  together   
3   (T95I, P681R)          0.166667           3  together   
6  (Y144-, P681H)          0.107143           3  together   
2   (T95I, L452R)          0.100000           3  together   
4  (Y144-, E484K)          0.093750           3  together   
7  (E484K, N501Y)          0.093750           3  together   

   individual_mut_count0  individual_mut_count1  
1                      5                      3  
5                      4                      4  
0                      5                      5  
3                      6                      3  
6                      4                      7  
2                      6                      5  
4                      4                      8  
7                      8                      4  
